# AutoUSD, sUSDe, Aave USDC and BIL (1-3 month T-Bill ETF) Comparison

Task:


> 
> Metrics:
> APY  
> Yield Volatility (annualized)  
> Excess vs 3M T-Bill  
> Information Ratio  
> For products:  
> USD: autoUSD, Ethena sUSDe, Aave USDC lending, US Gov short term treasuries  
> ETH: autoETH, Lido staking (stETH), Aave ETH lending  
> Time periods: 30 day, 1 year, since inception  
> *no 1 year on autoUSD since <1 year old  
> Definitions:  
> Yield Volatility: stdev of monthly net returns × √12  
> Excess vs Benchmark: difference between average net APY and the benchmark (3M T-bill for USD; Lido staking rate for ETH).  
> Information Ratio: average monthly excess return ÷ stdev of monthly excess returns, annualized  


# Methods


## Data sources:

Onchain calls on mainnet at the last block of each day

- autoUSD.converToAssets(1e18) / 1e6 -> USDC value of one autoUSD share
- sUSDe -> chainlink sUSDE -> USD oracle, safe USD value of sUSDe over time.
- Aave USDC lending, stataAaveUSDC.convertToAssets(1e6) / 1e6, growth of a share of the aave USDC vault

BIL `SPDR Bloomberg 1-3 Month T-Bill ETF`

https://www.ssga.com/us/en/intermediary/etfs/spdr-bloomberg-1-3-month-t-bill-etf-bil 

> The SPDR® Bloomberg  1-3 Month T-Bill ETF seeks to provide investment results that, before fees and expenses, correspond generally to the price and yield performance of the Bloomberg  1-3 Month U.S. Treasury Bill Index (the "Index")  
> Seeks to provide exposure to publicly issued U.S. Treasury Bills that have a remaining maturities between 1 and 3 months  
> Short duration fixed income is less exposed to fluctuations in interest rates than longer duration securities  
> Rebalanced on the last business day of the month  


As of Oct 26, 2025 BIL has ~42.5B AUM

```
!pip install yfinance
import yfinance as yf
tickers = ['BIL']
df = yf.download(tickers, start="2024-10-26", actions=True)
df.columns = df.columns.get_level_values(0)
df.to_csv('BIL data year to date.csv')

```



In [1]:
from multicall import Call

import plotly.express as px
import pandas as pd
import numpy as np

from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
)
from mainnet_launch.constants import ETH_CHAIN, AUTO_USD


# https://etherscan.io/address/0x73edDFa87C71ADdC275c2b9890f5c3a8480bC9E6#readProxyContract
# chainlink safe price of sUSDe over time
# https://etherscan.io/address/0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c aave USDC
# https://data.chain.link/feeds/ethereum/mainnet/susde-usd


end_of_day_oct_26_2024_block = 21053236
sUSDe_price_oracle = "0xFF3BC18cCBd5999CE63E788A1c250a88626aD099"
sUSDe = "0x9D39A5DE30e57443BfF2A8307A4256c8797A3497"  # mainnet
stata_aave = "0x73edDFa87C71ADdC275c2b9890f5c3a8480bC9E6"  # convert to shares calls

aave_convert_to_shares_call = Call(
    stata_aave,
    ["convertToAssets(uint256)(uint256)", int(1e6)],
    [("aave_convert_to_assets", safe_normalize_6_with_bool_success)],
)


sUSDe_convert_to_shares_call = Call(
    sUSDe,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [("sUSDe_convert_to_assets", safe_normalize_with_bool_success)],
)

autoUSD_nav_per_share_call = Call(
    AUTO_USD.autopool_eth_addr,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [("autoUSD_nav_per_share", safe_normalize_6_with_bool_success)],
)


def _process_chainlink_usd_price(success, args):
    """
    Normalize Chainlink latestRoundData -> price as float (answer / 1e6).
    args is: (roundId:uint80, answer:int256, startedAt:uint256, updatedAt:uint256, answeredInRound:uint80)
    """
    round_id, answer, started_at, updated_at, answered_in_round = args
    if success:
        return float(answer) / 1e8


chainlink_sUSDe_usd_price_call = Call(
    sUSDe_price_oracle,
    ["latestRoundData()((uint80,int256,uint256,uint256,uint80))"],
    [("sUSDe_usd_price", _process_chainlink_usd_price)],
)


blocks = build_blocks_to_use(ETH_CHAIN, start_block=end_of_day_oct_26_2024_block)

df = get_raw_state_by_blocks(
    calls=[
        aave_convert_to_shares_call,
        autoUSD_nav_per_share_call,
        chainlink_sUSDe_usd_price_call,
    ],
    blocks=blocks,
    chain=ETH_CHAIN,
)


bill_df = pd.read_csv("BIL data year to date.csv", parse_dates=["Date"], index_col="Date")
bill_df["BIL_close_price"] = bill_df["Close"]
bill_df.index = bill_df.index.date
df.index = df.index.date
large_df = df.merge(bill_df["BIL_close_price"], left_index=True, right_index=True, how="left")
large_df["BIL_close_price"] = large_df["BIL_close_price"].interpolate("linear")
start_date = pd.to_datetime("2025-05-01").date()
end_date = pd.to_datetime("2025-12-31").date()

end_of_day_prices = large_df[(large_df.index >= start_date) & (large_df.index <= end_date)].copy()
end_of_day_prices.index = pd.to_datetime(end_of_day_prices.index)

/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/eth_abi/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
end_of_day_prices

,aave_convert_to_assets,autoUSD_nav_per_share,sUSDe_usd_price,BIL_close_price
2025-05-01,1.127611,1.003976,1.168147,88.945229
2025-05-02,1.127702,1.004222,1.167726,88.974403
2025-05-03,1.127793,1.004526,1.167965,88.974403
2025-05-04,1.127886,1.004709,1.167646,88.974403
2025-05-05,1.127979,1.004807,1.169055,88.974403
...,...,...,...,...
2025-12-27,1.157201,1.057504,1.211743,91.360001
2025-12-28,1.157300,1.057746,1.211376,91.360001
2025-12-29,1.157396,1.057954,1.211954,91.360001
2025-12-30,1.157492,1.058041,1.212445,91.370003


In [ ]:
combined_df

# APY, Excess APY 30 days and since start

In [2]:
from mainnet_launch.adhoc.returns_math import (
    compute_series_apy,
    compute_most_recent_30_days_apy,
    compute_yield_volatility_annualized,
    compute_excess_vs_benchmark_apy,
    compute_information_ratio,
    compute_30_day_excess_vs_benchmark_apy,
)

start_date = end_of_day_prices.index.min()
end_date = end_of_day_prices.index.max()
thrity_days_before_end = end_date - pd.Timedelta(days=30)
all_time_apy = compute_series_apy(end_of_day_prices)
most_recent_30_days_apy = compute_most_recent_30_days_apy(end_of_day_prices)
yield_volatility = compute_yield_volatility_annualized(end_of_day_prices)
excess_apy_vs_benchmark = compute_excess_vs_benchmark_apy(end_of_day_prices, "BIL_close_price")
most_recent_excess_vs_benchmark = compute_30_day_excess_vs_benchmark_apy(end_of_day_prices, "BIL_close_price")
information_ratio = compute_information_ratio(end_of_day_prices, "BIL_close_price")

combined_df = pd.concat(
    [
        all_time_apy.rename("All Time APY"),
        most_recent_30_days_apy.rename("30 Day APY"),
        yield_volatility.rename("Yield Volatility"),
        excess_apy_vs_benchmark.rename("Excess APY vs Benchmark"),
        most_recent_excess_vs_benchmark.rename("30 Day Excess vs Benchmark"),
        information_ratio.rename("Information Ratio"),
    ],
    axis=1,
)

combined_df = combined_df.reindex(
    [
        "BIL_close_price",
        "autoUSD_nav_per_share",
        "aave_convert_to_assets",
        "sUSDe_usd_price",
    ]
)
combined_df = combined_df.round(2)

combined_df["All Time APY"] = combined_df["All Time APY"].apply(lambda x: str(x) + "%" if pd.notnull(x) else x)
combined_df["30 Day APY"] = combined_df["30 Day APY"].apply(lambda x: str(x) + "%" if pd.notnull(x) else x)
combined_df["Excess APY vs Benchmark"] = combined_df["Excess APY vs Benchmark"].apply(
    lambda x: str(x) + "%" if pd.notnull(x) else x
)
combined_df["30 Day Excess vs Benchmark"] = combined_df["30 Day Excess vs Benchmark"].apply(
    lambda x: str(x) + "%" if pd.notnull(x) else x
)
combined_df["Yield Volatility"] = combined_df["Yield Volatility"].apply(lambda x: str(x) + "%" if pd.notnull(x) else x)
print(f"Data from {start_date.date()} to {end_date.date()}")
print(combined_df.to_markdown())

Data from 2025-05-01 to 2025-12-31
|                        | All Time APY   | 30 Day APY   | Yield Volatility   | Excess APY vs Benchmark   | 30 Day Excess vs Benchmark   |   Information Ratio |
|:-----------------------|:---------------|:-------------|:-------------------|:--------------------------|:-----------------------------|--------------------:|
| BIL_close_price        | 4.12%          | 4.0%         | 0.1%               | 0.0%                      | 0.0%                         |              nan    |
| autoUSD_nav_per_share  | 8.2%           | 6.71%        | 0.44%              | 4.08%                     | 2.71%                        |                3.09 |
| aave_convert_to_assets | 4.0%           | 3.33%        | 0.11%              | -0.12%                    | -0.67%                       |               -0.1  |
| sUSDe_usd_price        | 5.74%          | 6.52%        | 0.67%              | 1.62%                     | 2.52%                        |                0.52 |

# older data

Data from 2025-05-01 to 2025-10-31
|                        | All Time APY   | 30 Day APY   | Yield Volatility   | Excess APY vs Benchmark   | 30 Day Excess vs Benchmark   |   Information Ratio |
|:-----------------------|:---------------|:-------------|:-------------------|:--------------------------|:-----------------------------|--------------------:|
| BIL_close_price        | 4.23%          | 4.06%        | 0.09%              | 0.0%                      | 0.0%                         |              nan    |
| autoUSD_nav_per_share  | 8.7%           | 8.08%        | 0.39%              | 4.47%                     | 4.02%                        |                3.73 |
| aave_convert_to_assets | 4.07%          | 3.81%        | 0.08%              | -0.16%                    | -0.25%                       |               -0.17 |
| sUSDe_usd_price        | 5.73%          | 3.3%         | 0.79%              | 1.5%                      | -0.76%                       |                0.33 |